<a href="https://colab.research.google.com/github/gahyunparkk/crosswalk_system/blob/main/yolo_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [ ]:
!mkdir -p /content/yolov5/data/scripts
# COCO2YOLO.py
!wget -O /content/yolov5/data/scripts/COCO2YOLO.py https://raw.githubusercontent.com/alexmihalyk23/COCO2YOLO/3d04bf84e6252166d04ed465c58ee481a421e68e/COCO2YOLO.py

In [ ]:
# COCO 데이터셋
!curl -L "http://images.cocodataset.org/zips/train2017.zip" > train2017.zip
!curl -L "http://images.cocodataset.org/zips/val2017.zip" > val2017.zip
!curl -L "http://images.cocodataset.org/annotations/annotations_trainval2017.zip" > annotations_trainval2017.zip

!unzip -q train2017.zip -d datasets/
!unzip -q val2017.zip -d datasets/
!unzip -q annotations_trainval2017.zip -d datasets/

In [ ]:
import json
from pathlib import Path
import os

train_annotations_path = 'datasets/annotations/instances_train2017.json'
val_annotations_path = 'datasets/annotations/instances_val2017.json'
train_images_dir = 'datasets/train2017'
val_images_dir = 'datasets/val2017'

vehicle_class_ids = [2, 3, 4, 6, 8]

def filter_coco_annotations(input_path, output_path, class_ids):
    with open(input_path) as f:
        data = json.load(f)

    filtered_images = []
    filtered_annotations = []
    image_ids = set()

    for annotation in data['annotations']:
        if annotation['category_id'] in class_ids:
            filtered_annotations.append(annotation)
            image_ids.add(annotation['image_id'])

    for image in data['images']:
        if image['id'] in image_ids:
            filtered_images.append(image)

    filtered_data = {
        'images': filtered_images,
        'annotations': filtered_annotations,
        'categories': [cat for cat in data['categories'] if cat['id'] in class_ids]
    }

    with open(output_path, 'w') as f:
        json.dump(filtered_data, f)

    return image_ids

def delete_unnecessary_images(image_dir, keep_image_ids):
    for image_file in os.listdir(image_dir):
        image_id = int(Path(image_file).stem.lstrip('0'))
        if image_id not in keep_image_ids:
            os.remove(os.path.join(image_dir, image_file))

train_keep_ids = filter_coco_annotations(train_annotations_path, 'datasets/annotations/filtered_train2017.json', vehicle_class_ids)
val_keep_ids = filter_coco_annotations(val_annotations_path, 'datasets/annotations/filtered_val2017.json', vehicle_class_ids)

delete_unnecessary_images(train_images_dir, train_keep_ids)
delete_unnecessary_images(val_images_dir, val_keep_ids)


In [ ]:
!python data/scripts/COCO2YOLO.py -j datasets/annotations/filtered_train2017.json -o datasets/yolo_format/train
!python data/scripts/COCO2YOLO.py -j datasets/annotations/filtered_val2017.json -o datasets/yolo_format/val

In [ ]:
!mv /content/yolov5/datasets/yolo_format/val/*.txt /content/yolov5/datasets/val2017/
!mv /content/yolov5/datasets/yolo_format/train/*.txt /content/yolov5/datasets/train2017/

In [ ]:
%%writefile data/vehicle.yaml
train: datasets/train2017
val: datasets/val2017

nc: 4  # 클래스 수
names: ['bicycle', 'car', 'motorcycle', 'bus', 'truck']  # 클래스 이름

In [ ]:
!python train.py --img 640 --batch 16 --epochs 100 --data data/vehicle.yaml --weights yolov5s.pt